# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Train Dataset

In [2]:
Train_dataset = pd.read_csv('train.csv',index_col='PassengerId')
Train_dataset.drop(columns=['Name','Ticket','Cabin'],inplace=True)

In [3]:
Train_dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


# Data Preprocessing(Training Set)

In [4]:
Train_dataset.shape

(891, 8)

In [5]:
Train_dataset.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
Train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [7]:
#Split data to X and y
X = Train_dataset.iloc[:, 1:].values
y = Train_dataset.iloc[:, 0].values

In [8]:
# Encode categorical data(Male-1,Female-0).
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])

In [9]:
# Handle Missing Values
from sklearn.impute import SimpleImputer
Mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')#Mean
Median_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')#Median
Mean_imputer.fit(X[:, 2:6])
X[:, 2:6] = Mean_imputer.transform(X[:, 2:6])
Median_imputer.fit(X[:, 2:7])
X[:, 2:7] = Median_imputer.transform(X[:, 2:7])

In [10]:
print(X)

[[3 1 22.0 ... 0.0 7.25 'S']
 [1 0 38.0 ... 0.0 71.2833 'C']
 [3 0 26.0 ... 0.0 7.925 'S']
 ...
 [3 0 29.69911764705882 ... 2.0 23.45 'S']
 [1 1 26.0 ... 0.0 30.0 'C']
 [3 1 32.0 ... 0.0 7.75 'Q']]


In [11]:
# Creating Dummy Variables
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Splitting Data

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [13]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Load Test Dataset

In [14]:
Test_dataset = pd.read_csv('test.csv',index_col='PassengerId')
Test_dataset.drop(columns=['Name','Ticket','Cabin'],inplace=True)

In [15]:
Test_dataset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34.5,0,0,7.8292,Q
893,3,female,47.0,1,0,7.0000,S
894,2,male,62.0,0,0,9.6875,Q
895,3,male,27.0,0,0,8.6625,S
896,3,female,22.0,1,1,12.2875,S


# Data Preprocessing (Test Set)

In [16]:
Test_dataset.shape

(418, 7)

In [17]:
Test_dataset.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [18]:
Test_set = Test_dataset.iloc[:,:].values

In [19]:
Test_set[:, 1] = le.fit_transform(Test_set[:, 1])
Mean_imputer.fit(Test_set[:, 2:6])
Test_set[:, 2:6] = Mean_imputer.transform(Test_set[:, 2:6])
Median_imputer.fit(Test_set[:, 2:7])
Test_set[:, 2:7] = Median_imputer.transform(Test_set[:, 2:7])

In [20]:
Test_set = np.array(ct.fit_transform(Test_set))
print(Test_set)

[[0.0 1.0 0.0 ... 0.0 0.0 7.8292]
 [0.0 0.0 1.0 ... 1.0 0.0 7.0]
 [0.0 1.0 0.0 ... 0.0 0.0 9.6875]
 ...
 [0.0 0.0 1.0 ... 0.0 0.0 7.25]
 [0.0 0.0 1.0 ... 0.0 0.0 8.05]
 [1.0 0.0 0.0 ... 1.0 1.0 22.3583]]


# Training a Model

### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
Logistic = LogisticRegression()
Logistic.fit(X_train, y_train)

D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Predict a Model
y_pred = Logistic.predict(X_test)

In [23]:
#Confusion Matrix,Accuracy Score

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[90 16]
 [19 54]]


0.8044692737430168

In [24]:
#Cross val score

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = Logistic, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 79.51 %
Standard Deviation: 3.25 %


D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\DATA_SCIENCE\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\DATA_SCIENCE

### SVM

In [25]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', random_state = 0)
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [26]:
y_pred = svm.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[90 16]
 [24 49]]


0.776536312849162

In [27]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = svm, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 78.95 %
Standard Deviation: 3.78 %


### Kernel SVM

In [28]:
from sklearn.svm import SVC
kernelsvm = SVC(kernel = 'rbf',gamma=0.2,C=0.5)
kernelsvm.fit(X_train, y_train)
y_pred = kernelsvm.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[100   6]
 [ 34  39]]


0.776536312849162

In [29]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = kernelsvm, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 83.72 %
Standard Deviation: 3.91 %


### KNN

In [30]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[100   6]
 [ 31  42]]


0.7932960893854749

In [31]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 82.17 %
Standard Deviation: 2.51 %


### Decision Tree

In [32]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[90 16]
 [22 51]]


0.7877094972067039

In [33]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = tree, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 77.68 %
Standard Deviation: 4.04 %


### Naive Bayes

In [34]:
from sklearn.naive_bayes import GaussianNB
Naive = GaussianNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[86 20]
 [28 45]]


0.7318435754189944

In [35]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = Naive, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 79.38 %
Standard Deviation: 4.25 %


### Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier
Random = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 1)
Random.fit(X_train, y_train)
y_pred = Random.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[99  7]
 [32 41]]


0.7821229050279329

In [37]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = Random, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 84.42 %
Standard Deviation: 2.65 %


### XGBoost

In [38]:
from xgboost import XGBClassifier
boost = XGBClassifier()
boost.fit(X_train, y_train)
y_pred = boost.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[96 10]
 [29 44]]


0.7821229050279329

In [39]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = boost, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 81.06 %
Standard Deviation: 3.27 %


### Cat Boost

In [40]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()
cat.fit(X_train, y_train)
y_pred = cat.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

Learning rate set to 0.008911
0:	learn: 0.6868117	total: 92.5ms	remaining: 1m 32s
1:	learn: 0.6801528	total: 94.4ms	remaining: 47.1s
2:	learn: 0.6743533	total: 96.2ms	remaining: 32s
3:	learn: 0.6688980	total: 98ms	remaining: 24.4s
4:	learn: 0.6631016	total: 99.9ms	remaining: 19.9s
5:	learn: 0.6571249	total: 102ms	remaining: 16.8s
6:	learn: 0.6511863	total: 103ms	remaining: 14.6s
7:	learn: 0.6465028	total: 104ms	remaining: 12.9s
8:	learn: 0.6409778	total: 106ms	remaining: 11.7s
9:	learn: 0.6355761	total: 107ms	remaining: 10.6s
10:	learn: 0.6301098	total: 109ms	remaining: 9.79s
11:	learn: 0.6257289	total: 110ms	remaining: 9.08s
12:	learn: 0.6201959	total: 112ms	remaining: 8.49s
13:	learn: 0.6155049	total: 113ms	remaining: 7.98s
14:	learn: 0.6104704	total: 115ms	remaining: 7.54s
15:	learn: 0.6058601	total: 116ms	remaining: 7.15s
16:	learn: 0.6017596	total: 117ms	remaining: 6.79s
17:	learn: 0.5973533	total: 119ms	remaining: 6.47s
18:	learn: 0.5925483	total: 120ms	remaining: 6.2s
19:	learn:

170:	learn: 0.3775280	total: 338ms	remaining: 1.64s
171:	learn: 0.3770777	total: 339ms	remaining: 1.63s
172:	learn: 0.3766698	total: 341ms	remaining: 1.63s
173:	learn: 0.3762802	total: 342ms	remaining: 1.62s
174:	learn: 0.3759952	total: 344ms	remaining: 1.62s
175:	learn: 0.3755819	total: 345ms	remaining: 1.61s
176:	learn: 0.3750047	total: 347ms	remaining: 1.61s
177:	learn: 0.3744775	total: 348ms	remaining: 1.61s
178:	learn: 0.3742330	total: 350ms	remaining: 1.6s
179:	learn: 0.3739035	total: 351ms	remaining: 1.6s
180:	learn: 0.3736820	total: 353ms	remaining: 1.59s
181:	learn: 0.3733826	total: 354ms	remaining: 1.59s
182:	learn: 0.3730604	total: 355ms	remaining: 1.59s
183:	learn: 0.3729336	total: 357ms	remaining: 1.58s
184:	learn: 0.3726942	total: 358ms	remaining: 1.58s
185:	learn: 0.3724236	total: 360ms	remaining: 1.57s
186:	learn: 0.3720086	total: 361ms	remaining: 1.57s
187:	learn: 0.3717806	total: 362ms	remaining: 1.56s
188:	learn: 0.3713727	total: 364ms	remaining: 1.56s
189:	learn: 0.

387:	learn: 0.3310020	total: 667ms	remaining: 1.05s
388:	learn: 0.3308851	total: 668ms	remaining: 1.05s
389:	learn: 0.3308322	total: 670ms	remaining: 1.05s
390:	learn: 0.3307304	total: 672ms	remaining: 1.05s
391:	learn: 0.3307146	total: 673ms	remaining: 1.04s
392:	learn: 0.3304686	total: 674ms	remaining: 1.04s
393:	learn: 0.3302090	total: 676ms	remaining: 1.04s
394:	learn: 0.3300149	total: 677ms	remaining: 1.04s
395:	learn: 0.3299126	total: 678ms	remaining: 1.03s
396:	learn: 0.3298749	total: 680ms	remaining: 1.03s
397:	learn: 0.3297822	total: 682ms	remaining: 1.03s
398:	learn: 0.3297077	total: 683ms	remaining: 1.03s
399:	learn: 0.3295405	total: 685ms	remaining: 1.03s
400:	learn: 0.3294245	total: 686ms	remaining: 1.02s
401:	learn: 0.3291925	total: 688ms	remaining: 1.02s
402:	learn: 0.3291272	total: 689ms	remaining: 1.02s
403:	learn: 0.3288606	total: 691ms	remaining: 1.02s
404:	learn: 0.3285890	total: 692ms	remaining: 1.02s
405:	learn: 0.3285195	total: 694ms	remaining: 1.01s
406:	learn: 

632:	learn: 0.3028488	total: 1.01s	remaining: 587ms
633:	learn: 0.3025827	total: 1.01s	remaining: 585ms
634:	learn: 0.3025472	total: 1.01s	remaining: 584ms
635:	learn: 0.3024152	total: 1.02s	remaining: 582ms
636:	learn: 0.3023409	total: 1.02s	remaining: 580ms
637:	learn: 0.3022800	total: 1.02s	remaining: 579ms
638:	learn: 0.3022656	total: 1.02s	remaining: 577ms
639:	learn: 0.3022407	total: 1.02s	remaining: 575ms
640:	learn: 0.3021168	total: 1.02s	remaining: 574ms
641:	learn: 0.3020483	total: 1.02s	remaining: 572ms
642:	learn: 0.3020400	total: 1.03s	remaining: 570ms
643:	learn: 0.3019759	total: 1.03s	remaining: 568ms
644:	learn: 0.3018971	total: 1.03s	remaining: 567ms
645:	learn: 0.3018201	total: 1.03s	remaining: 565ms
646:	learn: 0.3017262	total: 1.03s	remaining: 563ms
647:	learn: 0.3015866	total: 1.03s	remaining: 562ms
648:	learn: 0.3015840	total: 1.03s	remaining: 560ms
649:	learn: 0.3015367	total: 1.04s	remaining: 558ms
650:	learn: 0.3014532	total: 1.04s	remaining: 557ms
651:	learn: 

848:	learn: 0.2798598	total: 1.34s	remaining: 238ms
849:	learn: 0.2797980	total: 1.34s	remaining: 236ms
850:	learn: 0.2797270	total: 1.34s	remaining: 235ms
851:	learn: 0.2795328	total: 1.34s	remaining: 233ms
852:	learn: 0.2794984	total: 1.34s	remaining: 232ms
853:	learn: 0.2794734	total: 1.34s	remaining: 230ms
854:	learn: 0.2793747	total: 1.35s	remaining: 228ms
855:	learn: 0.2793487	total: 1.35s	remaining: 227ms
856:	learn: 0.2792438	total: 1.35s	remaining: 225ms
857:	learn: 0.2790571	total: 1.35s	remaining: 224ms
858:	learn: 0.2788896	total: 1.35s	remaining: 222ms
859:	learn: 0.2788203	total: 1.35s	remaining: 221ms
860:	learn: 0.2787431	total: 1.36s	remaining: 219ms
861:	learn: 0.2786370	total: 1.36s	remaining: 217ms
862:	learn: 0.2784588	total: 1.36s	remaining: 216ms
863:	learn: 0.2782656	total: 1.36s	remaining: 214ms
864:	learn: 0.2781641	total: 1.36s	remaining: 213ms
865:	learn: 0.2781080	total: 1.36s	remaining: 211ms
866:	learn: 0.2780158	total: 1.36s	remaining: 210ms
867:	learn: 

0.7877094972067039

In [41]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = cat, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.008515
0:	learn: 0.6867952	total: 4.01ms	remaining: 4s
1:	learn: 0.6833800	total: 5.07ms	remaining: 2.53s
2:	learn: 0.6793673	total: 6.96ms	remaining: 2.31s
3:	learn: 0.6738210	total: 8.69ms	remaining: 2.16s
4:	learn: 0.6678059	total: 10.4ms	remaining: 2.07s
5:	learn: 0.6622004	total: 12.2ms	remaining: 2.02s
6:	learn: 0.6562559	total: 13.9ms	remaining: 1.97s
7:	learn: 0.6509298	total: 15.6ms	remaining: 1.93s
8:	learn: 0.6454340	total: 17.3ms	remaining: 1.91s
9:	learn: 0.6403555	total: 19.2ms	remaining: 1.9s
10:	learn: 0.6357558	total: 21.1ms	remaining: 1.89s
11:	learn: 0.6311951	total: 22.9ms	remaining: 1.88s
12:	learn: 0.6268917	total: 24.6ms	remaining: 1.87s
13:	learn: 0.6215132	total: 26.3ms	remaining: 1.85s
14:	learn: 0.6170200	total: 27.8ms	remaining: 1.83s
15:	learn: 0.6135406	total: 28.9ms	remaining: 1.78s
16:	learn: 0.6089942	total: 30.5ms	remaining: 1.76s
17:	learn: 0.6053160	total: 32ms	remaining: 1.75s
18:	learn: 0.6005839	total: 33.5ms	remaining: 1.73

229:	learn: 0.3552030	total: 338ms	remaining: 1.13s
230:	learn: 0.3548122	total: 339ms	remaining: 1.13s
231:	learn: 0.3546824	total: 341ms	remaining: 1.13s
232:	learn: 0.3544528	total: 342ms	remaining: 1.13s
233:	learn: 0.3541094	total: 343ms	remaining: 1.12s
234:	learn: 0.3539254	total: 345ms	remaining: 1.12s
235:	learn: 0.3535613	total: 346ms	remaining: 1.12s
236:	learn: 0.3535162	total: 347ms	remaining: 1.12s
237:	learn: 0.3533155	total: 349ms	remaining: 1.12s
238:	learn: 0.3530701	total: 350ms	remaining: 1.11s
239:	learn: 0.3527829	total: 352ms	remaining: 1.11s
240:	learn: 0.3526663	total: 353ms	remaining: 1.11s
241:	learn: 0.3524469	total: 355ms	remaining: 1.11s
242:	learn: 0.3520806	total: 356ms	remaining: 1.11s
243:	learn: 0.3520351	total: 358ms	remaining: 1.11s
244:	learn: 0.3518543	total: 359ms	remaining: 1.11s
245:	learn: 0.3516482	total: 360ms	remaining: 1.1s
246:	learn: 0.3514942	total: 362ms	remaining: 1.1s
247:	learn: 0.3511862	total: 363ms	remaining: 1.1s
248:	learn: 0.3

466:	learn: 0.3159220	total: 675ms	remaining: 770ms
467:	learn: 0.3158564	total: 677ms	remaining: 770ms
468:	learn: 0.3157880	total: 679ms	remaining: 768ms
469:	learn: 0.3157066	total: 680ms	remaining: 767ms
470:	learn: 0.3156465	total: 682ms	remaining: 766ms
471:	learn: 0.3155868	total: 683ms	remaining: 764ms
472:	learn: 0.3153106	total: 685ms	remaining: 763ms
473:	learn: 0.3152508	total: 686ms	remaining: 761ms
474:	learn: 0.3150912	total: 688ms	remaining: 761ms
475:	learn: 0.3149917	total: 690ms	remaining: 759ms
476:	learn: 0.3147879	total: 691ms	remaining: 758ms
477:	learn: 0.3146743	total: 693ms	remaining: 757ms
478:	learn: 0.3146438	total: 694ms	remaining: 755ms
479:	learn: 0.3144564	total: 696ms	remaining: 754ms
480:	learn: 0.3144197	total: 697ms	remaining: 753ms
481:	learn: 0.3143270	total: 699ms	remaining: 751ms
482:	learn: 0.3142919	total: 700ms	remaining: 749ms
483:	learn: 0.3140972	total: 702ms	remaining: 748ms
484:	learn: 0.3139684	total: 703ms	remaining: 747ms
485:	learn: 

690:	learn: 0.2895243	total: 1s	remaining: 449ms
691:	learn: 0.2894641	total: 1.01s	remaining: 448ms
692:	learn: 0.2893928	total: 1.01s	remaining: 446ms
693:	learn: 0.2892397	total: 1.01s	remaining: 445ms
694:	learn: 0.2891142	total: 1.01s	remaining: 444ms
695:	learn: 0.2890756	total: 1.01s	remaining: 442ms
696:	learn: 0.2890041	total: 1.01s	remaining: 441ms
697:	learn: 0.2887797	total: 1.01s	remaining: 439ms
698:	learn: 0.2886693	total: 1.02s	remaining: 438ms
699:	learn: 0.2885277	total: 1.02s	remaining: 436ms
700:	learn: 0.2884163	total: 1.02s	remaining: 435ms
701:	learn: 0.2884040	total: 1.02s	remaining: 434ms
702:	learn: 0.2883423	total: 1.02s	remaining: 432ms
703:	learn: 0.2881338	total: 1.02s	remaining: 431ms
704:	learn: 0.2880391	total: 1.02s	remaining: 429ms
705:	learn: 0.2879846	total: 1.03s	remaining: 428ms
706:	learn: 0.2878014	total: 1.03s	remaining: 426ms
707:	learn: 0.2876408	total: 1.03s	remaining: 425ms
708:	learn: 0.2874644	total: 1.03s	remaining: 423ms
709:	learn: 0.2

918:	learn: 0.2670024	total: 1.34s	remaining: 118ms
919:	learn: 0.2669468	total: 1.34s	remaining: 116ms
920:	learn: 0.2669072	total: 1.34s	remaining: 115ms
921:	learn: 0.2668643	total: 1.34s	remaining: 114ms
922:	learn: 0.2667985	total: 1.34s	remaining: 112ms
923:	learn: 0.2665690	total: 1.34s	remaining: 111ms
924:	learn: 0.2664761	total: 1.35s	remaining: 109ms
925:	learn: 0.2663447	total: 1.35s	remaining: 108ms
926:	learn: 0.2661320	total: 1.35s	remaining: 106ms
927:	learn: 0.2659883	total: 1.35s	remaining: 105ms
928:	learn: 0.2658996	total: 1.35s	remaining: 103ms
929:	learn: 0.2658498	total: 1.35s	remaining: 102ms
930:	learn: 0.2658359	total: 1.35s	remaining: 100ms
931:	learn: 0.2657490	total: 1.36s	remaining: 99ms
932:	learn: 0.2656586	total: 1.36s	remaining: 97.5ms
933:	learn: 0.2655638	total: 1.36s	remaining: 96.1ms
934:	learn: 0.2654907	total: 1.36s	remaining: 94.6ms
935:	learn: 0.2653295	total: 1.36s	remaining: 93.1ms
936:	learn: 0.2652829	total: 1.36s	remaining: 91.7ms
937:	lea

...

209:	learn: 0.3640831	total: 323ms	remaining: 1.22s
210:	learn: 0.3638480	total: 325ms	remaining: 1.21s
211:	learn: 0.3636420	total: 327ms	remaining: 1.21s
212:	learn: 0.3632841	total: 329ms	remaining: 1.21s
213:	learn: 0.3630797	total: 330ms	remaining: 1.21s
214:	learn: 0.3629556	total: 332ms	remaining: 1.21s
215:	learn: 0.3627750	total: 334ms	remaining: 1.21s
216:	learn: 0.3625785	total: 336ms	remaining: 1.21s
217:	learn: 0.3622842	total: 339ms	remaining: 1.22s
218:	learn: 0.3619456	total: 341ms	remaining: 1.21s
219:	learn: 0.3617703	total: 343ms	remaining: 1.22s
220:	learn: 0.3615335	total: 345ms	remaining: 1.22s
221:	learn: 0.3611991	total: 347ms	remaining: 1.22s
222:	learn: 0.3608171	total: 349ms	remaining: 1.22s
223:	learn: 0.3607512	total: 351ms	remaining: 1.21s
224:	learn: 0.3604999	total: 352ms	remaining: 1.21s
225:	learn: 0.3602191	total: 354ms	remaining: 1.21s
226:	learn: 0.3600240	total: 357ms	remaining: 1.22s
227:	learn: 0.3596260	total: 359ms	remaining: 1.22s
228:	learn: 

387:	learn: 0.3290342	total: 635ms	remaining: 1s
388:	learn: 0.3289558	total: 637ms	remaining: 1s
389:	learn: 0.3288455	total: 638ms	remaining: 998ms
390:	learn: 0.3287129	total: 640ms	remaining: 996ms
391:	learn: 0.3286192	total: 641ms	remaining: 994ms
392:	learn: 0.3283428	total: 643ms	remaining: 992ms
393:	learn: 0.3282028	total: 644ms	remaining: 991ms
394:	learn: 0.3278707	total: 645ms	remaining: 989ms
395:	learn: 0.3277901	total: 647ms	remaining: 987ms
396:	learn: 0.3276536	total: 649ms	remaining: 985ms
397:	learn: 0.3275765	total: 650ms	remaining: 984ms
398:	learn: 0.3274038	total: 652ms	remaining: 982ms
399:	learn: 0.3273261	total: 654ms	remaining: 981ms
400:	learn: 0.3272425	total: 655ms	remaining: 978ms
401:	learn: 0.3271252	total: 656ms	remaining: 976ms
402:	learn: 0.3270563	total: 658ms	remaining: 974ms
403:	learn: 0.3267298	total: 659ms	remaining: 973ms
404:	learn: 0.3266259	total: 661ms	remaining: 971ms
405:	learn: 0.3265384	total: 662ms	remaining: 969ms
406:	learn: 0.3263

587:	learn: 0.3040455	total: 955ms	remaining: 669ms
588:	learn: 0.3038769	total: 957ms	remaining: 668ms
589:	learn: 0.3037201	total: 958ms	remaining: 666ms
590:	learn: 0.3036190	total: 960ms	remaining: 664ms
591:	learn: 0.3035422	total: 961ms	remaining: 663ms
592:	learn: 0.3034250	total: 963ms	remaining: 661ms
593:	learn: 0.3032603	total: 965ms	remaining: 659ms
594:	learn: 0.3031945	total: 966ms	remaining: 658ms
595:	learn: 0.3030795	total: 968ms	remaining: 656ms
596:	learn: 0.3030065	total: 969ms	remaining: 654ms
597:	learn: 0.3029137	total: 971ms	remaining: 653ms
598:	learn: 0.3026586	total: 973ms	remaining: 651ms
599:	learn: 0.3023951	total: 974ms	remaining: 649ms
600:	learn: 0.3023264	total: 976ms	remaining: 648ms
601:	learn: 0.3023005	total: 978ms	remaining: 647ms
602:	learn: 0.3022286	total: 979ms	remaining: 645ms
603:	learn: 0.3020849	total: 981ms	remaining: 643ms
604:	learn: 0.3018723	total: 982ms	remaining: 641ms
605:	learn: 0.3016322	total: 984ms	remaining: 640ms
606:	learn: 

795:	learn: 0.2816231	total: 1.28s	remaining: 328ms
796:	learn: 0.2815740	total: 1.28s	remaining: 327ms
797:	learn: 0.2814509	total: 1.28s	remaining: 325ms
798:	learn: 0.2813403	total: 1.29s	remaining: 324ms
799:	learn: 0.2812215	total: 1.29s	remaining: 322ms
800:	learn: 0.2810828	total: 1.29s	remaining: 320ms
801:	learn: 0.2809966	total: 1.29s	remaining: 319ms
802:	learn: 0.2808864	total: 1.29s	remaining: 317ms
803:	learn: 0.2807880	total: 1.29s	remaining: 316ms
804:	learn: 0.2806847	total: 1.29s	remaining: 314ms
805:	learn: 0.2805447	total: 1.3s	remaining: 312ms
806:	learn: 0.2804462	total: 1.3s	remaining: 311ms
807:	learn: 0.2803000	total: 1.3s	remaining: 309ms
808:	learn: 0.2801506	total: 1.3s	remaining: 308ms
809:	learn: 0.2798930	total: 1.3s	remaining: 306ms
810:	learn: 0.2797945	total: 1.3s	remaining: 304ms
811:	learn: 0.2797242	total: 1.31s	remaining: 303ms
812:	learn: 0.2795883	total: 1.31s	remaining: 301ms
813:	learn: 0.2794697	total: 1.31s	remaining: 299ms
814:	learn: 0.2794

995:	learn: 0.2610014	total: 1.6s	remaining: 6.43ms
996:	learn: 0.2607889	total: 1.6s	remaining: 4.83ms
997:	learn: 0.2607487	total: 1.6s	remaining: 3.22ms
998:	learn: 0.2607009	total: 1.61s	remaining: 1.61ms
999:	learn: 0.2606090	total: 1.61s	remaining: 0us
Accuracy: 83.00 %
Standard Deviation: 2.59 %


### ANN

In [42]:
import tensorflow as tf

In [43]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1,  activation='sigmoid'))

In [44]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 1, epochs = 100)

Train on 712 samples
Epoch 1/100
712/712 [==============================] - 1s 2ms/sample - loss: 0.6284 - accuracy: 0.6601
Epoch 2/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.4802 - accuracy: 0.79780s - l
Epoch 3/100
712/712 [==============================] - 1s 979us/sample - loss: 0.4437 - accuracy: 0.8090
Epoch 4/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.4288 - accuracy: 0.8174
Epoch 5/100
712/712 [==============================] - 1s 995us/sample - loss: 0.4181 - accuracy: 0.8230
Epoch 6/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.4116 - accuracy: 0.8202
Epoch 7/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.4037 - accuracy: 0.8188
Epoch 8/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.4010 - accuracy: 0.8357
Epoch 9/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.3967 - accuracy: 0.8399
Epoch 10/100
712/712 [====================

712/712 [==============================] - 1s 985us/sample - loss: 0.3539 - accuracy: 0.8596
Epoch 78/100
712/712 [==============================] - 1s 974us/sample - loss: 0.3537 - accuracy: 0.8666
Epoch 79/100
712/712 [==============================] - 1s 949us/sample - loss: 0.3521 - accuracy: 0.8567
Epoch 80/100
712/712 [==============================] - ETA: 0s - loss: 0.3491 - accuracy: 0.86 - 1s 987us/sample - loss: 0.3508 - accuracy: 0.8624
Epoch 81/100
712/712 [==============================] - 1s 980us/sample - loss: 0.3522 - accuracy: 0.8596
Epoch 82/100
712/712 [==============================] - 1s 980us/sample - loss: 0.3519 - accuracy: 0.8596 - loss: 0.3473 - accuracy: 0.
Epoch 83/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.3533 - accuracy: 0.8581
Epoch 84/100
712/712 [==============================] - 1s 972us/sample - loss: 0.3510 - accuracy: 0.8539
Epoch 85/100
712/712 [==============================] - 1s 1ms/sample - loss: 0.3508 - accuracy

In [45]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[100   6]
 [ 30  43]]
0.7988826815642458
